In [1]:
import os
import ee
import geemap
from dotenv import load_dotenv
import psycopg2 as pg
import pandas as pd
import geopandas as gpd
from shapely import wkt
import time

In [2]:
load_dotenv()

usr=os.getenv('usuario')
pss=os.getenv('contra')
lectura_sap=os.getenv('sap')

engine_postgis = pg.connect(f"dbname='postgis' user='{usr}' host='10.225.101.5' port='5432' password={pss}")
ee.Initialize()
#ee.Authenticate()

In [93]:
# Load the feature collection
fc = ee.FeatureCollection('users/raulperezastorga/predios_actualizado')

# Get the maximum value of a property in the feature collection
max_value = fc.reduceColumns(ee.Reducer.max(), ['objectid']).get('max')

mxgee=max_value.getInfo()
mxgee


20781

In [94]:
consulta=f'''select
	objectid,
	idarea,
	idpredio,
	idtiporegimendominio as idtiporegi,
	superficieha as superficie,
	estado_id,
	estado_vigencia as estado_vig,
	idpredio || '-' || nombre as etiqueta,
	nombre,
	porcen_propiedad as porcen_pro,
	referencia,
	ultimorodal as ultimoroda,
	wkt
from
	raul.predios p
where objectid > {mxgee}'''

In [95]:
df=pd.read_sql(consulta, engine_postgis)

In [102]:
asset_to_update='users/raulperezastorga/predios'
asset_name = 'users/raulperezastorga/predios_nuevos'

In [103]:
if df['objectid'].count()==0:
    print('Limite Predial actualizado')
else:
    df['wkt'] = df['wkt'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='wkt', crs= 'EPSG:4326')
    fc = geemap.geopandas_to_ee(gdf)
    # Save the merged feature collection as a new asset
    
    ee.data.deleteAsset(asset_name)

    task = ee.batch.Export.table.toAsset(
        collection=fc, 
        description='PrediosNuevos',
        assetId=asset_name)
    task.start()
    time.sleep(300) #sleep for 5 minutes



Limite Predial actualizado


In [5]:
uso='''select
	objectid,
	case
		when tipouso in ('EUGL', 'EUNI', 'EHNG', 'PIRA') then tipouso || extract(year from CURRENT_DATE) - anoplantacion
	end as pk,
	idpredio,
	idrodal,
	tipouso,
	anoplantacion as apl,
	fechainicio as fini,
	fechafin as ffin,
	wkt
from
	raul.usoactual u 
where objectid > 355095521'''

In [6]:
df=pd.read_sql(uso, engine_postgis)

In [7]:
df['wkt'] = df['wkt'].apply(wkt.loads)

In [8]:
gdf = gpd.GeoDataFrame(df, geometry='wkt', crs= 'EPSG:4326')

In [86]:
fc = geemap.geopandas_to_ee(gdf)

In [87]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [88]:
Map.addLayer(fc, {}, "geopandas to ee")


In [89]:

# Define the existing assets to merge
fc1 = ee.FeatureCollection('users/raulperezastorga/predios')

# Merge the two feature collections
merged_fc = fc1.merge(fc)

# Save the merged feature collection as a new asset
asset_name = 'users/raulperezastorga/predios_actualizado'
task = ee.batch.Export.table.toAsset(
    collection=merged_fc, 
    description='PrediosActualizados',
    assetId=asset_name)
task.start()


In [99]:
# delete asset
ee.data.deleteAsset('users/raulperezastorga/predios')


In [100]:
asset_name = 'users/raulperezastorga/predios'
task = ee.batch.Export.table.toAsset(
    collection=ee.FeatureCollection('users/raulperezastorga/predios_actualizado'), 
    description='PrediosActualizados',
    assetId=asset_name)
task.start()

In [9]:
# Export GeoDataFrame to Shapefile
gdf.to_file('uso.shp')

DriverSupportError: ESRI Shapefile does not support datetime fields

In [ ]:
import zipfile

# list of files to zip
files = ['uso.shp', 'uso.dbf', 'uso.shx', 'uso.cpg']

# create a zip file object
with zipfile.ZipFile('uso.zip', 'w') as myzip:

    # add each file to the zip file
    for file in files:
        # add the file to the zip file with a modified name
        myzip.write(file)



In [61]:
# Replace the path with your local path where shapefile is stored
zip_path = 'uso.zip'
asset_name = 'users/raulperezastorga/uso'

# Upload ZIP file to GEE Assets
geemap.upload_to_ee(zip_path, asset_name)


AttributeError: module 'geemap' has no attribute 'upload_to_ee'

In [7]:
gdf = gpd.GeoDataFrame(df, geometry='geom')
gdf1=gdf[gdf['idpredio']==71268]

In [37]:
shapefile_path = 'geofiles/predios.shp'

# Load the shapefile as an EE feature collection
fc = ee.FeatureCollection('users/rauledop/predios')

# Convert the shapefile to an EE asset
asset_id = 'users/rauledop/predios'


ee.data.upload(shapefile_path, asset_id=asset_id)



AttributeError: module 'ee.data' has no attribute 'upload'

In [9]:
import json
ee_fc=ee.FeatureCollection(json.loads(gdf.to_json()))


In [10]:
asset_id='users/rauledop/predios'

In [11]:
ee.data.createAsset({'type': 'FeatureCollection', 'features': ee_fc.getInfo()['features']}, asset_id)


EEException: Request payload size exceeds the limit: 10485760 bytes.

In [26]:
geemap.ee_search()